In [2]:
import pandas as pd
import numpy as np


In [12]:
bike_data = pd.read_csv('../data/santander_locations.csv')
bike_data.head()

,Station.Id,StationName,longitude,latitude,Easting,Northing
0,1,"River Street, Clerkenwell",-0.109971,51.5292,531202.520,182832.020
1,2,"Phillimore Gardens, Kensington",-0.197574,51.4996,525207.070,179391.860
2,3,"Christopher Street, Liverpool Street",-0.084606,51.5213,532984.810,182001.530
3,4,"St. Chad's Street, King's Cross",-0.120974,51.5301,530436.760,182911.990
4,5,"Sedding Street, Sloane Square",-0.156876,51.4931,528051.649,178742.097


In [ ]:
bike_data